# Семинар по теме «`async`»

## Блокирующий семинар

**Как влияет на уровень**

Чтобы подтвердить продвинутый уровень, необходимо решить задачи и сдать на проверку ноутбук.

**Как влияет на оценку**

За выполнение заданий **всех** семинаров (блокирующих и неблокирующих) начисляются баллы. Чем больше баллов, тем выше оценка за курс.

## Оценивание работы на семинаре

**Система оценивания —** бинарная:

  - если все задачи решены корректно, без ошибок и полностью соответствуют стандартам кода на курсе, то задание выполнено и оценка — **10 баллов**;
  - если решения содержат ошибки или не соответствуют требованиям, то задание не выполнено и оценка — **0 баллов**.


**Проверка задания**

- Перед тем как сдать задание, убедись, что твой код работает без ошибок и соответствует стандартам. Для этого используй автоматическую проверку.
- Загрузи задание на LMS. Ассистент проверит, соответствуют ли твои решения требованиям и целям задания, и выставит оценку.


**Доработка**

- Если твоё задание получило 0 баллов, его вернут на доработку через LMS с комментариями о том, что нужно исправить.

Твоя задача — реализовать асинхронную систему для чтения, обработки и сохранения больших объёмов данных из CSV-файла, который содержит рецензии на игры. Данные обрабатываются по частям (чанкам), агрегируются по играм и сохраняются в отдельные CSV-файлы для каждой игры.

Данные для задания нужно скачать по [ссылке из Яндекс.Диска ЦУ](https://disk.yandex.ru/d/JnMz7hjln5NAng)

### Шаг 1
Создай класс для чтения файла CSV в чанках, чтобы обрабатывать его частями, не загружая весь файл в память.

#### Создание класса `CSVReader`

   - Атрибуты:
     - `file_path` — путь к файлу CSV.
     - `chunk_size` — размер чанка данных для чтения (по умолчанию 1024 байта).
     - `headers` — список заголовков из CSV-файла.

   - Методы:
     - `__init__(self, file_path, chunk_size=1024)` — инициализирует объект с указанием пути к файлу и размера чанка.
     
     - `read_in_chunks(self)` — асинхронно читает файл CSV по частям, разбивает файл на строки и возвращает их по мере чтения.
         - Ожидаемый вывод: каждая порция данных будет возвращена в виде списка словарей, где ключами являются заголовки CSV-файла.

     - `parse_chunk(self, chunk)` — принимает строку с набором данных, разбивает её с использованием `csv.DictReader` и возвращает список словарей, соответствующих данным в каждой строке.

     - `clean_review(self, review)` — очищает строку данных, убирая лишние пробелы и форматируя поле `username` с учётом возможных многострочных записей.


#### Примеры использования

```python
async def main():
    csv_reader = CSVReader('steam_game_reviews.csv', chunk_size=2048)

    async for chunk in csv_reader.read_in_chunks():
        pprint(chunk)  # Печать каждой порции данных

# Запуск асинхронной программы
asyncio.run(main())
```

#### Ожидаемый вывод
   - Если данные успешно считаны, то выводятся:
     ```python
     строки, соответствующие чанкам данных
     ```
   - Если файл пуст или данные не найдены:
     ```
     []
     ```

### Шаг 2

Cоздай класс для обработки рецензий. Этот класс будет агрегировать рецензии по играм, чтобы собирать все данные об игре в одном месте.

#### Создание класса `GameDataProcessor`
   - Атрибуты:
     - `game_data` — словарь, в котором ключами будут названия игр, а значениями — списки рецензий.

   - Методы:
     - `__init__(self)` — инициализирует объект с пустым словарём для хранения данных по играм.
     
     - `process_review(self, review)` — добавляет рецензию в список соответствующей игры. Если игра ещё не встречалась, то создаётся новый список для неё.
       - Ожидаемый вывод: данные рецензии добавляются в словарь, где ключ — это название игры.

     - `get_all_game_data(self)` — возвращает полный словарь с данными по всем играм.
       - Ожидаемый вывод: возвращает словарь с играми и списками рецензий.


#### Примеры использования

```python
# Пример обработки рецензий
processor = GameDataProcessor()

review1 = {'game_name': 'Game A', 'review': 'Awesome!', 'rating': '5'}
review2 = {'game_name': 'Game B', 'review': 'Not bad', 'rating': '4'}
review3 = {'game_name': 'Game A', 'review': 'Loved it!', 'rating': '5'}

# Обработка рецензий
processor.process_review(review1)
processor.process_review(review2)
processor.process_review(review3)

# Получение данных по играм
game_data = processor.get_all_game_data()
pprint(game_data)
```

#### Ожидаемый вывод
```python
{
   'Game A': [
       {'game_name': 'Game A', 'review': 'Awesome!', 'rating': '5'},
       {'game_name': 'Game A', 'review': 'Loved it!', 'rating': '5'},
   ],
   'Game B': [
       {'game_name': 'Game B', 'review': 'Not bad', 'rating': '4'},
   ],
}
```

### Шаг 3

Реализуй класс для асинхронного сохранения данных по играм в отдельные CSV-файлы. Каждая игра будет сохранена в свой файл, содержащий информацию о рецензиях.

#### Cоздание класса `GameFileWriter`
   - Атрибуты:
     - `output_dir` — директория для сохранения файлов с данными по играм.

   - Методы:
     - `__init__(self, output_dir)` — инициализирует объект с указанием директории для сохранения файлов и создаёт директорию, если она не существует.
     
     - `save_game_data(self, game_name, reviews)` — асинхронно сохраняет данные о рецензиях по игре в отдельный CSV-файл. Каждая строка файла будет представлять одну рецензию.
       - Ожидаемый вывод: CSV-файл будет сохранён в указанной директории с данными по рецензиям на игру.


#### Примеры использования

```python
# Пример сохранения данных в CSV
file_writer = GameFileWriter("output_data")

game_reviews = [
    {'review': 'Great game!', 'hours_played': '10', 'helpful': '5', 'funny': '1', 'recommendation': 'Yes', 'date': '2024-10-12', 'username': 'user1', 'products_in_account': 100},
    {'review': 'Not bad', 'hours_played': '8', 'helpful': '3', 'funny': '0', 'recommendation': 'No', 'date': '2024-10-11', 'username': 'user2', 'products_in_account': 50},
]

# Асинхронное сохранение рецензий в CSV-файл
await file_writer.save_game_data('Game_A', game_reviews)
```
#### Ожидаемый вывод
- В директории `output_data` создаётся файл `Game_A.csv` со следующим содержимым:
     ```
     review,hours_played,helpful,funny,recommendation,date,username,products_in_account
     Great game!,10,5,1,Yes,2024-10-12,user1,100
     Not bad,8,3,0,No,2024-10-11,user2,50
     ```

### Шаг 4
Cоздай класс для управления чтением, обработкой и сохранением рецензий. Класс `ReviewManager` объединяет все предыдущие шаги, организуя работу с большими данными и разделяя их по играм для дальнейшего сохранения.


####  Создание класса `ReviewManager`
   - Атрибуты:
     - `csv_reader` — экземпляр класса `CSVReader` для чтения данных из CSV-файла.
     - `data_processor` — экземпляр класса `GameDataProcessor` для обработки рецензий.
     - `file_writer` — экземпляр класса `GameFileWriter` для сохранения данных по играм в отдельные файлы.
   
   - Методы:
     - `__init__(self, csv_path, output_dir, chunk_size=10000, total_lines=909740)` — инициализирует экземпляры классов для чтения данных, обработки рецензий и их сохранения.
     
     - `process_reviews(self)` — управляет чтением рецензий из файла, обработкой каждой рецензии и сохранением данных по играм в CSV-файлы.
       - Ожидаемый вывод: печатаются сообщения о процессе обработки чанков данных и записи рецензий для каждой игры.

     - `_write_game_data(self, game, reviews, index, total_games)` — сохраняет данные по конкретной игре в файл, выводит сообщения о процессе записи данных.


#### Примеры использования

```python
# Пример использования ReviewManager для чтения и обработки рецензий
review_manager = ReviewManager('reviews.csv', 'output_data')

# Асинхронная обработка рецензий
await review_manager.process_reviews()
```

#### Ожидаемый вывод
   - Во время обработки данных:
     ```
     Processing chunk 1
     Processing chunk 2
     ...
     ```
   - Во время записи данных:
     ```
     Writing data for 'Game A' (1 / 50)...
     Complete 'Game A' (1 / 50)...
     Writing data for 'Game B' (2 / 50)...
     Complete 'Game B' (2 / 50)...
     ...
     ```